In [ ]:
import argparse
import os
import random
import time
from distutils.util import strtobool

import gym
import numpy as np
import pygame
from gym_macro_overcooked.macActEnvWrapper import MacEnvWrapper
from llm_utils import *

rewardList = {"subtask finished": 10, "correct delivery": 200, "wrong delivery": -5, "step penalty": -0.1}
gird_dim=[7,7]
env = gym.make('Dynamic-v3', grid_dim=gird_dim, task=3, rewardList=rewardList, map_type = "Dynamic", n_agent = 1, obs_radius = 2, mode = "vector", debug = True)
env.reset()




ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5220:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM default


In [ ]:
ITEMIDX= {"space": 0, "counter": 1, "agent": 2, "tomato": 3, "lettuce": 4, "plate": 5, "knife": 6, "delivery": 7, "onion": 8}
def map2prompt(agent,map):
    map_text = []
    for i in range(len(map)):
        map_text.append(" ".join([str(x) for x in map[i]]))
    map_text = "\n".join(map_text)

    text=('以上是当前环境地图，不同的数字代表不同的物品，具体如下：\n'+
            str(ITEMIDX)+'\n'+
          #'目前的任务列表是：'+str(task_list)+'\n'+
          '其中你的位置是：'+str((agent.x,agent.y))+'\n'+
          '请你给我一段提示词，让我能够使用这个提示词让LLM帮助完成任务。')
    return map_text+'\n'+text
    
    

In [ ]:
new_obs, reward, done, info = env.run(0)
test=env.render()
obs2text(new_obs,task=env.task)

{'prompt': "There are two fixed cutting boards in the room. You notice a lettuce and an onion on the different tables. Currently you are carrying an unchopped tomato in hand. To serve the dish of a bowl only containing chopped tomato and lettuce, let's think step by step, you should first",
 'action': ['pick up the tomato',
  'pick up the lettuce',
  'pick up the onion',
  'take the empty bowl',
  'put the tomato on the first cutting board',
  'put the tomato on the second cutting board',
  'serve the dish',
  'chop nothing']}

In [ ]:
done=False
obs=env.reset()
while done == False:
    obs_text = obs2text(obs,task=env.task)
    messages = create_message(obs_text)
    response = call_gpt(messages,model='gpt-4')
    print(response)
    action = extract_and_check_first_number(response.content,len(obs_text["action"]))
    new_obs, reward, done, info = env.run(action)
    print(reward,done,info)
    test=env.render()
    obs = new_obs

ChatCompletionMessage(content='Action: 1. pick up the tomato\nReason: Since only chopped tomato and lettuce need to be served in the bowl, the first step in the process would include picking up a tomato.', role='assistant', function_call=None, tool_calls=None)
-0.199 False {'cur_mac': [3], 'mac_done': [True], 'collision': [], 'macro_action_steps': 2}
ChatCompletionMessage(content='Action: 5. \nReason: Because I am already holding the tomato, I need to place it on a cutting board before I can chop it. The onion is not required in the recipe so there is no need to pick it up. I will collect the lettuce later because both lettuce and tomato need to be chopped before serving and it makes more sense to complete one task before moving onto the next.', role='assistant', function_call=None, tool_calls=None)
-0.490099501 False {'cur_mac': [2], 'mac_done': [True], 'collision': [], 'macro_action_steps': 5}


KeyboardInterrupt: 

In [ ]:
done=False
obs=env.reset()


In [ ]:
obs_text = obs2text(obs,task=env.task)
messages = create_message(obs_text)
print('prompt:',messages[0]["content"])
response = call_gpt(messages,model='gpt-4o')
print('====================')
print('response:',response.content)
action = extract_and_check_first_number(response.content,len(obs_text["action"]))
new_obs, reward, done, info = env.run(action)
print('====================')
print('Info:',reward,done,info)
test=env.render()
obs = new_obs

prompt: There are two fixed cutting boards in the room. An unchopped tomato is on the first cutting board. Currently you are standing in front of the first cutting board without anything in hand. To serve the dish of a bowl only containing chopped tomato and lettuce, let's think step by step, you should first:
 Action: 1. pick up the tomato
2. pick up the lettuce
3. pick up the onion
4. take the empty bowl
5. walk to the first cutting board
6. walk to the second cutting board
7. serve nothing
8. chop the tomato
NOTE replay with the action number.
response: 1
Info: -0.1 False {'cur_mac': [4], 'mac_done': [True], 'collision': [], 'macro_action_steps': 1}


In [ ]:
obs,reward,done,info=env.step([2])
print(obs2text(obs[0],task=env.task))
a=env.render()

{'prompt': "There are two fixed cutting boards in the room. Currently you are carrying an unchopped tomato in hand. To serve the dish of a bowl only containing chopped tomato and lettuce, let's think step by step, you should first", 'action': ['pick up the tomato', 'pick up the lettuce', 'pick up the onion', 'take the empty bowl', 'put the tomato on the first cutting board', 'put the tomato on the second cutting board', 'serve the dish', 'chop nothing']}


In [ ]:
env.tomato[0].chopped

False

In [ ]:
env.agent[0].holding

In [ ]:
env.task

3

In [ ]:
env.knife[0].holding.chopped

'tomato'

In [ ]:
env.lettuce[0].is_cooked

AttributeError: 'Lettuce' object has no attribute 'is_cooked'

In [ ]:
b=env.agent[0].holding

In [ ]:
obs

([array([0.14285714, 0.        , 0.        , 0.14285714, 0.71428571,
         0.        , 0.28571429, 0.85714286, 0.        , 0.85714286,
         0.71428571, 0.14285714, 0.        , 0.28571429, 0.        ,
         0.42857143, 0.        , 0.14285714, 0.71428571, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        ])],
 [-0.1],
 False,
 {'cur_mac': [4], 'mac_done': [True], 'collision': []})

In [ ]:
obs,reward,done,info=env.run([6])
print(obs2text(obs,task=env.task))
a=env.render()

{'prompt': "There are two fixed cutting boards in the room. Currently you don't have anything in hand. To serve the dish of a bowl only containing chopped tomato and lettuce, let's think step by step, you should first", 'action': ['pick up the tomato', 'pick up the lettuce', 'pick up the onion', 'take the empty bowl', 'walk to the first cutting board', 'walk to the second cutting board', 'serve nothing', 'chop nothing']}


{'prompt': "There are two fixed cutting boards in the room. An unchopped tomato is on the first cutting board. Currently you are standing in front of the first cutting board without anything in hand. To serve the dish of a bowl only containing chopped tomato and lettuce, let's think step by step, you should first", 'action': ['pick up the tomato', 'pick up the lettuce', 'pick up the onion', 'take the empty bowl', 'walk to the first cutting board', 'walk to the second cutting board', 'serve nothing', 'chop the tomato']}


In [ ]:
import argparse
import os
import random
import time
from distutils.util import strtobool
import pygame
import gym
import numpy as np
from gym_macro_overcooked.macActEnvWrapper import MacEnvWrapper
from llm_utils import *

rewardList = {"subtask finished": 10, "correct delivery": 200, "wrong delivery": -5, "step penalty": -0.1}
gird_dim=[9,9]
env = gym.make('Dynamic-v3', grid_dim=gird_dim, task=3, rewardList=rewardList, map_type = "Dynamic", n_agent = 1, obs_radius = 2, mode = "vector", debug = True)
#env = gym.make('Overcooked-LLMA-v3', grid_dim=gird_dim, task=3, rewardList=rewardList, map_type = "A", n_agent = 1, obs_radius = 2, mode = "vector", debug = True)
env.reset()
#给我写一个将键盘WASD输入转换为动作的函数
def key2action(key):
    if key == 'd':
        return 0
    elif key == 's':
        return 1
    elif key == 'a':
        return 2
    elif key == 'w':
        return 3
    else:
        return None

#开始游戏，使用WASD控制移动
done=False
obs=env.reset()
while done == False:
    env.render()
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True
        elif event.type == pygame.KEYDOWN:
            key = pygame.key.name(event.key)
            action = key2action(key)
            if action is not None:
                new_obs, reward, done, info = env.step([action])
                print(f"Reward: {reward}, Done: {done}, Info: {info}")
                obs = new_obs



ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5220:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM default


Reward: [-0.1], Done: False, Info: {'cur_mac': [1], 'mac_done': [True], 'collision': []}
Reward: [-0.1], Done: False, Info: {'cur_mac': [0], 'mac_done': [True], 'collision': []}
Reward: [-0.1], Done: False, Info: {'cur_mac': [3], 'mac_done': [True], 'collision': []}
Reward: [-0.1], Done: False, Info: {'cur_mac': [0], 'mac_done': [True], 'collision': []}
Reward: [-0.1], Done: False, Info: {'cur_mac': [0], 'mac_done': [True], 'collision': []}
Reward: [-0.1], Done: False, Info: {'cur_mac': [0], 'mac_done': [True], 'collision': []}
Reward: [-0.1], Done: False, Info: {'cur_mac': [0], 'mac_done': [True], 'collision': []}
Reward: [-0.1], Done: False, Info: {'cur_mac': [0], 'mac_done': [True], 'collision': []}
Reward: [-0.1], Done: False, Info: {'cur_mac': [3], 'mac_done': [True], 'collision': []}
Reward: [-0.1], Done: False, Info: {'cur_mac': [1], 'mac_done': [True], 'collision': []}
Reward: [-0.1], Done: False, Info: {'cur_mac': [1], 'mac_done': [True], 'collision': []}
Reward: [-0.1], Done:

error: Couldn't open /home/bluelink/TWOSOME-main/gym_macro_overcooked/render/graphics/ChoppedTomato-ChoppedOnion.png

In [ ]:
import argparse
import os
import random
import time
from distutils.util import strtobool
import pygame
import gym
import numpy as np
from gym_macro_overcooked.macActEnvWrapper import MacEnvWrapper
from llm_utils import *

rewardList = {"subtask finished": 10, "correct delivery": 200, "wrong delivery": -5, "step penalty": -0.1}
gird_dim=[9,9]
env = gym.make('Dynamic-v3', grid_dim=gird_dim, task=3, rewardList=rewardList, map_type = "Dynamic", n_agent = 1, obs_radius = 2, mode = "vector", debug = True)


ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5220:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM default


In [ ]:
env.reset()

[array([0.        , 0.77777778, 0.        , 0.11111111, 0.88888889,
        0.        , 0.22222222, 0.88888889, 0.        , 0.88888889,
        0.77777778, 0.44444444, 0.55555556, 0.44444444, 0.77777778,
        0.44444444, 0.        , 0.11111111, 0.11111111, 0.22222222,
        0.        , 0.66666667, 0.        , 0.66666667, 0.88888889,
        0.        , 0.        , 0.        , 0.11111111, 0.88888889,
        0.11111111, 0.        , 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ])]

In [ ]:
env._apply_diff()

In [ ]:
env.render()

array([[[114,  93,  51],
        [114,  93,  51],
        [114,  93,  51],
        ...,
        [114,  93,  51],
        [114,  93,  51],
        [114,  93,  51]],

       [[114,  93,  51],
        [220, 170, 110],
        [220, 170, 110],
        ...,
        [220, 170, 110],
        [220, 170, 110],
        [114,  93,  51]],

       [[114,  93,  51],
        [220, 170, 110],
        [220, 170, 110],
        ...,
        [220, 170, 110],
        [220, 170, 110],
        [114,  93,  51]],

       ...,

       [[114,  93,  51],
        [220, 170, 110],
        [220, 170, 110],
        ...,
        [220, 170, 110],
        [220, 170, 110],
        [114,  93,  51]],

       [[114,  93,  51],
        [220, 170, 110],
        [220, 170, 110],
        ...,
        [220, 170, 110],
        [220, 170, 110],
        [114,  93,  51]],

       [[114,  93,  51],
        [114,  93,  51],
        [114,  93,  51],
        ...,
        [114,  93,  51],
        [114,  93,  51],
        [114,  93,  51]]

In [ ]:
env.plate[0].dirty

True

In [ ]:
import argparse
import os
import random
import time
from distutils.util import strtobool
import pygame
import gym
import numpy as np
from gym_macro_overcooked.macActEnvWrapper import MacEnvWrapper
from llm_utils import *

rewardList = {"subtask finished": 10, "correct delivery": 200, "wrong delivery": -5, "step penalty": -0.1}
gird_dim=[9,9]
env = gym.make('Dynamic-v3', task=3, rewardList=rewardList, n_agent = 1, obs_radius = 2, obs_mode = "vector", debug = True)
#env = gym.make('Overcooked-LLMA-v3', grid_dim=gird_dim, task=3, rewardList=rewardList, map_type = "A", n_agent = 1, obs_radius = 2, mode = "vector", debug = True)
env.reset()
#给我写一个将键盘WASD输入转换为动作的函数
def key2action(key):
    if key == 'd':
        return 0
    elif key == 's':
        return 1
    elif key == 'a':
        return 2
    elif key == 'w':
        return 3
    else:
        return None
#开始游戏，使用WASD控制移动
done=False
obs=env.reset()

while done == False:
    
    key = input("Please input a key: ")
    action = key2action(key)
    if action is not None:
        new_obs, reward, done, info = env.step([action])
        print(f"Reward: {reward}, Done: {done}, Info: {info}")

    env.render()


In [ ]:

contained_meat=["1"]
"-".join(contained_meat)
if contained_meat:
    print('yes')
else:
    print('no')

yes


In [ ]:
a=[1,2,3]
a[-2:]

[2, 3]

In [ ]:
class IngredientProcessingFSM:
    def __init__(self, target_item, env, agent):
        current_step_name = "idle"
        self.target_item = target_item
        self.agent = agent
        self.env=env
        self.target_name = target_item.rawName


In [ ]:
test=IngredientProcessingFSM(env.tomato[0],env,env.agent[0])

In [ ]:
class TopFSM:
    def __init__(self, env, agent_number):
        current_step_name = "idle"   #idle,processing,completed
        self.task = env.task
        self.env = env
        self.ingredients_veg = ["tomato", "lettuce", "onion"]
        self.ingredients_meat = ["steak"]
        self.agent = env.agent[agent_number]
        self.task_steps = []
        self.current_step = 0
        self.subtask = None
        self.create_step_list()

    def create_step_list(self):
        # Process vegetable ingredients
        for ing in self.ingredients_veg:
            if ing in self.task:
                self.task_steps.append(self.create_step_dict(ing, 'chopped'))
                self.task_steps.append({"target": env.plate[0], "name": 'plate', "is_done": env.plate[0].holding.rawname==ing})
                
        # Process meat ingredients
        for ing in self.ingredients_meat:
            if ing in self.task:
                self.task_steps.append(self.create_step_dict(ing, 'cooked'))
                self.task_steps.append({"target": env.plate[0], "name": 'plate', "is_done": env.plate[0].holding.rawname==ing})

    def create_step_dict(self, ing, state_attr):
        for target in self.env.itemDic[ing]:
            if getattr(target, state_attr):
                return {"target": target, "name": ing, "is_done": True}
        first_target = self.env.itemDic[ing][0]
        return {"target": first_target, "name": ing, "is_done": getattr(first_target, state_attr)}

    def reset_state(self):
        self.state = "idle"
        self.current_step = 0
        self.subtask = None
        self.task=env.task

    def update_step_and_creat_subtask(self):
        # Update the current step and create subtask
        for i in len(self.task_steps):
            step=self.task_steps[i]
            if step["is_done"] == False:
                self.current_step = i
                self.state = "processing"
                if step["name"] in self.ingredients_veg or step["name"] in self.ingredients_meat:
                    self.subtask = IngredientProcessingFSM(step["target"], self.env, self.agent)
                elif step["name"] == "plate":
                    self.subtask = CollectingFSM(self.task_steps[i-1]["target"], self.env, self.agent)
                    

    def chooes_macro_action(self):
        if self.subtask==None:
            self.create_step_list()
            self.update_step_and_creat_subtask()
        self.subtask.update_state()
        if self.subtask.state == "completed":
            self.create_step_list()
            self.update_step_and_creat_subtask()
        
        action_index = self.subtask.choose_macro_action(self.env.itemDic, self.env.Macro_actions)
        return action_index
            
            
            
            
            
class IngredientProcessingFSM:
    def __init__(self, target_item, env, agent):
        self.state = "idle"
        self.target_item = target_item
        self.agent = agent
        self.env=env
        self.target_name = target_item.rawName
        self.subtask_steps = []
        self.current_step = 0
        self.create_step_list()

    def check_target_position_in_item(self, target, items):
        return any(i.holding == target for i in items)

    def create_step_list(self):
        cutboard_process_ing = ["tomato", "lettuce", "onion"]
        cutboard_process = {
            "get ingredients": self.agent.holding == self.target_item,
            "go to knife": self.check_target_position_in_item(self.target_item, self.game_items['knife']),
            "chop": False
        }
        pan_process_ing = ["steak"]
        pan_process = {
            "get ingredients": self.agent.holding == self.target_item,
            "put into the pan": self.check_target_position_in_item(self.target_item, self.game_items['pan']),
            "wait": False
        }

        if self.target_name in cutboard_process_ing:
            self.subtask_steps = cutboard_process
        elif self.target_name in pan_process_ing:
            self.subtask_steps = pan_process
        else:
            raise ValueError("Ingredient not found")

    def update_state(self):
        self.create_step_list()
        if self.state == "idle":
            self.state = "peocessing"
        
        for i, (key, value) in enumerate(self.subtask_steps.items()):
            if value==False:
                self.current_step = i
                break
        if self.state=="processing":
            self.state = "completed"
        

    def choose_macro_action(self):
        Macro_actions=env.macroActionName
        current_step_name=self.subtask_steps.keys()[self.current_step]
        
        if current_step_name == "get ingredients":
            action_index = Macro_actions.index(f"get {self.target_name}")
        elif current_step_name == "go to knife":
            for knife in self.env.knife:
                if knife.holding is None:
                    action_index = Macro_actions.index(f"go to the knife {knife.id}")
                    break
        elif current_step_name == "chop":
            action_index = Macro_actions.index("chop")
        elif current_step_name == "put into the pan":
            for pan in self.env.pan:
                if pan.holding is None:
                    action_index = Macro_actions.index(f"go to the pan {pan.id}")
                    break
        elif current_step_name == "wait":
            action_index = 100
        else:
            action_index = 100
            print("No available action")
        
        return action_index

            
            
class CollectingFSM:
    def __init__(self, target_item,env,agent):
        self.state = "idle"
        self.target_item = target_item
        self.agent = agent
        self.env = env
        self.subtask_steps = []
        self.current_step = 0
        self.create_step_list()
        

    def create_step_list(self):
        return {
            "get plate": self.agent.holding.rawName == "plate",
            "get ingredients": False
        }
    
    def update_state(self):
        self.create_step_list()
        if self.state == "idle":
            self.state = "peocessing"
        
        for i, (key, value) in enumerate(self.subtask_steps.items()):
            if value==False:
                self.current_step = i
                break
        if self.state=="processing":
            self.state = "completed"
            
            
    def choose_macro_action(self):
        Macro_actions=env.macroActionName
        current_step_name=self.subtask_steps.keys()[self.current_step]
        
        if current_step_name == "idle":
            self.update_state()
        elif current_step_name == "get plate":
            for plate in self.env.plate:
                if plate.containing == None :
                    action_index = Macro_actions.index(f"get plate {plate.id}")
                    break
        elif current_step_name == "get ingredients":
            action_index = Macro_actions.index(f"get {self.target_item.rawName}")
        elif current_step_name == "wait":
            action_index = 100
        else:
            action_index = 100
            print("No available action")

        return action_index
            

In [ ]:
import argparse
import os
import random
import time
from distutils.util import strtobool
import pygame
import gym
import numpy as np
from gym_macro_overcooked.macActEnvWrapper import MacEnvWrapper
from llm_utils import *

rewardList = {"subtask finished": 10, "correct delivery": 200, "wrong delivery": -5, "step penalty": -0.1}
gird_dim=[7,7]
env = gym.make('Overcooked-LLMA-v3', grid_dim=gird_dim, task=3, rewardList=rewardList, map_type = "A", n_agent = 2, obs_radius = 2, mode = "vector", debug = True)

ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5220:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM default


In [ ]:
env.reset()

[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 3, 0, 1, 2, 1, 4, 0,
        0, 0, 1, 0, 0, 0]),
 array([0, 5, 0, 1, 6, 0, 2, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 4, 0,
        0, 0, 1, 0, 0, 0])]

In [ ]:
env.render()

array([[[114,  93,  51],
        [114,  93,  51],
        [114,  93,  51],
        ...,
        [114,  93,  51],
        [114,  93,  51],
        [114,  93,  51]],

       [[114,  93,  51],
        [220, 170, 110],
        [220, 170, 110],
        ...,
        [220, 170, 110],
        [220, 170, 110],
        [114,  93,  51]],

       [[114,  93,  51],
        [220, 170, 110],
        [220, 170, 110],
        ...,
        [220, 170, 110],
        [220, 170, 110],
        [114,  93,  51]],

       ...,

       [[114,  93,  51],
        [220, 170, 110],
        [220, 170, 110],
        ...,
        [220, 170, 110],
        [220, 170, 110],
        [114,  93,  51]],

       [[114,  93,  51],
        [220, 170, 110],
        [220, 170, 110],
        ...,
        [220, 170, 110],
        [220, 170, 110],
        [114,  93,  51]],

       [[114,  93,  51],
        [114,  93,  51],
        [114,  93,  51],
        ...,
        [114,  93,  51],
        [114,  93,  51],
        [114,  93,  51]]

In [ ]:
env.task

'lettuce-tomato salad'

In [ ]:
aifsm=TopFSM(env,0)

AttributeError: 'Plate' object has no attribute 'holding'

In [ ]:
Macro_actions=["get tomato", "get lettuce", "get onion", "get plate 1", "go to knife 1", "go to knife 2", "deliver", "chop"]
search_keyword="get"
index=[i for i, x in enumerate(Macro_actions) if x == search_keyword]

ValueError: 'get' is not in list

In [ ]:
process_state=["get ingredients","chop ingredients","put into the plate"]
state="get ingredients"

cutboard_process={"get ingredients":False,"chop ingredients":False,"put into the plate":False}
cutboard_process[0]

KeyError: 0

In [ ]:
# 示例游戏状态
game_state = {
    "has_tomato": False,
    "has_lettuce": False,
    "has_onion": False,
    "has_steak": False
}

In [ ]:

# 示例游戏状态
game_state = {
    "has_tomato": False,
    "has_lettuce": False,
    "has_onion": False,
    "has_steak": False
}

# 创建FSM并设置任务
fsm = FSM()
fsm.set_task("steak with lettuce and tomato and onion")

# 更新和执行状态
fsm.update_state(game_state)
action = fsm.perform_action(game_state)
print(f"Action: {action}")  # Collect steak

# 更新游戏状态以模拟任务进展
game_state["has_steak"] = True
fsm.update_state(game_state)
action = fsm.perform_action(game_state)
print(f"Action: {action}")  # Cook steak

game_state["has_tomato"] = True
game_state["has_lettuce"] = True
game_state["has_onion"] = True
fsm.update_state(game_state)
action = fsm.perform_action(game_state)
print(f"Action: {action}")  # Cut ingredients

game_state["has_tomato"] = False  # Simulate cutting vegetables
game_state["has_lettuce"] = False
game_state["has_onion"] = False
fsm.update_state(game_state)
action = fsm.perform_action(game_state)
print(f"Action: {action}")  # Serve


In [ ]:
top_fsm = TopFSM('tomato', env, env.agent[0])



In [ ]:
action = top_fsm.perform_action()
new_obs, reward, done, info = env.step([action])
top_fsm.update_state()

In [ ]:
map_setting = open("gym_macro_overcooked/maps/"+"MapC"+".yaml", "r")

In [ ]:
class Test:
    def __init__(self):
        self.a=1
        self.b=2
        self.c=3
    def __str__(self):
        return f"a:{self.a},b:{self.b},c:{self.c}"
    
A=Test()
b=A.b
A.b=A.b*2
c=A.c
c=c*2

In [ ]:
class Test:
    def __init__(self):
        self.values = {'a': 1, 'b': 2, 'c': 3}

    def __str__(self):
        return f"a:{self.values['a']}, b:{self.values['b']}, c:{self.values['c']}"

A = Test()
b = A.values['b']
A.values['b'] = A.values['b'] * 2
c = A.values['c']
A.values['c'] *= 2

print(A)  # 输出：a:1, b:4, c:6
print(b)  # 输出：2
print(c)  # 输出：3

a:1, b:4, c:6
2
3


In [ ]:
ITEMIDX = {
    "space": 0,
    "counter": 1,
    "agent": 2,
    "tomato": 3,
    "lettuce": 4,
    "plate": 5,
    "knife": 6,
    "delivery": 7,
    "onion": 8,
    "pan": 9,
    "steak": 10,
    "sink": 11,
    "trash_can": 12,
}
#get dict keys by list of values
list(ITEMIDX.keys())

['space',
 'counter',
 'agent',
 'tomato',
 'lettuce',
 'plate',
 'knife',
 'delivery',
 'onion',
 'pan',
 'steak',
 'sink',
 'trash_can']

In [ ]:
def text():
    print("no")
    return True

if text():
    print('yes')

no
yes


In [9]:
import argparse
import os
import random
import time
from distutils.util import strtobool
import pygame
import gym
import numpy as np
from gym_macro_overcooked.macActEnvWrapper import MacEnvWrapper

# from llm_utils import *

rewardList = {
    "subtask finished": 10,
    "correct delivery": 200,
    "wrong delivery": -5,
    "step penalty": -0.1,
}

env = gym.make(
    "Dynamic-v3",
    task=3,
    rewardList=rewardList,
    n_agent=1,
    obs_radius=2,
    obs_mode="vector",
    debug=True,
)
# env = gym.make('Overcooked-LLMA-v3', grid_dim=gird_dim, task=3, rewardList=rewardList, map_type = "A", n_agent = 1, obs_radius = 2, mode = "vector", debug = True)
env.reset()


# 给我写一个将键盘WASD输入转换为动作的函数
def key2action(key):
    if key == "d":
        return 0
    elif key == "s":
        return 1
    elif key == "a":
        return 2
    elif key == "w":
        return 3
    else:
        return None


# 开始游戏，使用WASD控制移动
done = False
# env._apply_diff()
obs = env.reset()
while done == False:
    env.render()
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True
        elif event.type == pygame.KEYDOWN:
            key = pygame.key.name(event.key)
            print(f"Key: {key}")
            action = key2action(key)
            if action is not None:
                new_obs, reward, done, info = env.step([action])
                print(f"Reward: {reward}, Done: {done}, Info: {info}")
                obs = new_obs


Key: s
Reward: [-0.1], Done: False, Info: {'cur_mac': [1], 'mac_done': [True], 'collision': []}
Key: a
Reward: [-0.1], Done: False, Info: {'cur_mac': [2], 'mac_done': [True], 'collision': []}
Key: w
Reward: [-0.1], Done: False, Info: {'cur_mac': [3], 'mac_done': [True], 'collision': []}
Key: d
Reward: [-0.1], Done: False, Info: {'cur_mac': [0], 'mac_done': [True], 'collision': []}
Key: s
Reward: [-0.1], Done: False, Info: {'cur_mac': [1], 'mac_done': [True], 'collision': []}
Key: a
Reward: [-0.1], Done: False, Info: {'cur_mac': [2], 'mac_done': [True], 'collision': []}
Key: w
Reward: [-0.1], Done: False, Info: {'cur_mac': [3], 'mac_done': [True], 'collision': []}
Key: d
Reward: [-0.1], Done: False, Info: {'cur_mac': [0], 'mac_done': [True], 'collision': []}
Key: d
Reward: [-0.1], Done: False, Info: {'cur_mac': [0], 'mac_done': [True], 'collision': []}
Key: a
Reward: [-0.1], Done: False, Info: {'cur_mac': [2], 'mac_done': [True], 'collision': []}
Key: a
Reward: [-0.1], Done: False, Info

: 

In [1]:
import argparse
import os
import random
import time
from distutils.util import strtobool
import pygame
import gym
import numpy as np
from gym_macro_overcooked.macActEnvWrapper import MacEnvWrapper

# from llm_utils import *

rewardList = {
    "subtask finished": 10,
    "correct delivery": 200,
    "wrong delivery": -5,
    "step penalty": -0.1,
    "burned penalty": -2,
}

env = gym.make(
    "Dynamic-v3",
    n_task=2,
    rewardList=rewardList,
    n_agent=1,
    obs_radius=2,
    obs_mode="vector",
    debug=True,
)
# env = gym.make('Overcooked-LLMA-v3', grid_dim=gird_dim, task=3, rewardList=rewardList, map_type = "A", n_agent = 1, obs_radius = 2, mode = "vector", debug = True)
#env.reset()

pygame 2.5.2 (SDL 2.28.3, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
env.reset()

[array([0.77777778, 0.88888889, 0.55555556, 0.88888889, 0.66666667,
        0.88888889, 0.33333333, 0.88888889, 0.        , 0.55555556,
        0.        , 0.77777778, 0.44444444, 0.        , 0.11111111,
        0.77777778, 0.88888889, 0.55555556, 0.11111111, 0.88888889,
        0.88888889, 0.77777778, 0.88888889, 0.11111111, 0.        ,
        0.        , 0.        , 0.11111111, 0.        , 0.22222222,
        0.        , 0.33333333, 0.        , 0.44444444, 0.        ,
        0.55555556, 0.        , 0.66666667, 0.        , 0.77777778,
        0.        , 0.88888889, 0.11111111, 0.        , 0.11111111,
        0.33333333, 0.11111111, 0.77777778, 0.11111111, 0.88888889,
        0.22222222, 0.        , 0.22222222, 0.33333333, 0.22222222,
        0.88888889, 0.33333333, 0.        , 0.33333333, 0.33333333,
        0.33333333, 0.88888889, 0.44444444, 0.        , 0.44444444,
        0.33333333, 0.44444444, 0.44444444, 0.44444444, 0.77777778,
        0.44444444, 0.88888889, 0.55555556, 0.  

In [5]:
env.tasks

[{'ingredients': ('tomato', 'onion', 'steak'), 'task_encoding': [1, 0, 1, 1]},
 {'ingredients': ('lettuce',), 'task_encoding': [0, 1, 0, 0]}]

In [6]:
env.eventManager.tasks

[{'ingredients': ('tomato',), 'task_encoding': [1, 0, 0, 0]},
 {'ingredients': ('tomato', 'onion'), 'task_encoding': [1, 0, 1, 0]}]

In [6]:
env.reset()

[array([0.77777778, 0.88888889, 0.55555556, 0.88888889, 0.66666667,
        0.88888889, 0.33333333, 0.88888889, 0.        , 0.55555556,
        0.        , 0.77777778, 0.44444444, 0.        , 0.11111111,
        0.77777778, 0.88888889, 0.55555556, 0.11111111, 0.88888889,
        0.88888889, 0.77777778, 0.88888889, 0.11111111, 0.        ,
        0.        , 0.        , 0.11111111, 0.        , 0.22222222,
        0.        , 0.33333333, 0.        , 0.44444444, 0.        ,
        0.55555556, 0.        , 0.66666667, 0.        , 0.77777778,
        0.        , 0.88888889, 0.11111111, 0.        , 0.11111111,
        0.33333333, 0.11111111, 0.77777778, 0.11111111, 0.88888889,
        0.22222222, 0.        , 0.22222222, 0.33333333, 0.22222222,
        0.88888889, 0.33333333, 0.        , 0.33333333, 0.33333333,
        0.33333333, 0.88888889, 0.44444444, 0.        , 0.44444444,
        0.33333333, 0.44444444, 0.44444444, 0.44444444, 0.77777778,
        0.44444444, 0.88888889, 0.55555556, 0.  

In [7]:
env.tasks

[{'ingredients': ('lettuce', 'steak'), 'task_encoding': [0, 1, 0, 1]},
 {'ingredients': ('lettuce',), 'task_encoding': [0, 1, 0, 0]}]

In [8]:
env.taskManager.tasks

[{'ingredients': ('lettuce', 'steak'), 'task_encoding': [0, 1, 0, 1]},
 {'ingredients': ('lettuce',), 'task_encoding': [0, 1, 0, 0]}]

In [9]:
env.render()

array([[[114,  93,  51],
        [114,  93,  51],
        [114,  93,  51],
        ...,
        [114,  93,  51],
        [114,  93,  51],
        [114,  93,  51]],

       [[114,  93,  51],
        [220, 170, 110],
        [220, 170, 110],
        ...,
        [220, 170, 110],
        [220, 170, 110],
        [114,  93,  51]],

       [[114,  93,  51],
        [220, 170, 110],
        [220, 170, 110],
        ...,
        [220, 170, 110],
        [220, 170, 110],
        [114,  93,  51]],

       ...,

       [[114,  93,  51],
        [220, 170, 110],
        [220, 170, 110],
        ...,
        [220, 170, 110],
        [220, 170, 110],
        [114,  93,  51]],

       [[114,  93,  51],
        [220, 170, 110],
        [220, 170, 110],
        ...,
        [220, 170, 110],
        [220, 170, 110],
        [114,  93,  51]],

       [[114,  93,  51],
        [114,  93,  51],
        [114,  93,  51],
        ...,
        [114,  93,  51],
        [114,  93,  51],
        [114,  93,  51]]

In [7]:
import argparse
import os
import random
import time
from distutils.util import strtobool
import pygame
import gym
import numpy as np
from gym_macro_overcooked.macActEnvWrapper import MacEnvWrapper

rewardList = {
    "subtask finished": 10,
    "correct delivery": 200,
    "wrong delivery": -5,
    "step penalty": -0.1,
    "burned penalty": -2,
}

env = gym.make(
    "Dynamic-v3",
    n_task=2,
    rewardList=rewardList,
    n_agent=1,
    obs_radius=2,
    obs_mode="vector",
    debug=True,
)
env.reset()


# 将键盘WASD输入转换为动作的函数
def key2action(key):
    if key == "d":
        return 0
    elif key == "s":
        return 1
    elif key == "a":
        return 2
    elif key == "w":
        return 3
    else:
        return None


# 开始游戏，使用WASD控制移动
done = False
obs = env.reset()

# 初始化pygame
pygame.init()
screen = pygame.display.set_mode((600, 400))

# 控制渲染和事件处理的频率
clock = pygame.time.Clock()

while not done:
    env.render()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True
        elif event.type == pygame.KEYDOWN:
            key = pygame.key.name(event.key)
            print(f"Key: {key}")
            action = key2action(key)
            if action is not None:
                new_obs, reward, done, info = env.step([action])
                print(f"Reward: {reward}, Done: {done}, Info: {info}")
                obs = new_obs

    # 控制每秒钟的帧率
    clock.tick(30)

pygame.quit()


Key: s
Reward: [-0.1], Done: False, Info: {'cur_mac': [1], 'mac_done': [True], 'collision': []}
Key: s
Reward: [-0.1], Done: False, Info: {'cur_mac': [1], 'mac_done': [True], 'collision': []}
Key: s
Reward: [-0.1], Done: False, Info: {'cur_mac': [1], 'mac_done': [True], 'collision': []}
Key: s
Reward: [-0.1], Done: False, Info: {'cur_mac': [1], 'mac_done': [True], 'collision': []}
Key: a
Reward: [-0.1], Done: False, Info: {'cur_mac': [2], 'mac_done': [True], 'collision': []}
Key: w
Reward: [-0.1], Done: False, Info: {'cur_mac': [3], 'mac_done': [True], 'collision': []}


KeyboardInterrupt: 

In [11]:
obs,reward,done,info=env.step([0])
task_text ="task:"+str([str(task["ingredients"]) for task in env.tasks])
reward_text = "reward:"+str(reward)
env.game.display_info([task_text,reward_text])
env.render()
print(obs, reward, done, info)

[array([0.77777778, 0.88888889, 0.55555556, 0.88888889, 0.66666667,
       0.88888889, 0.33333333, 0.88888889, 0.        , 0.55555556,
       0.        , 0.77777778, 0.44444444, 0.        , 0.11111111,
       0.77777778, 0.88888889, 0.55555556, 0.11111111, 0.88888889,
       0.88888889, 0.77777778, 0.88888889, 0.11111111, 0.        ,
       0.        , 0.        , 0.11111111, 0.        , 0.22222222,
       0.        , 0.33333333, 0.        , 0.44444444, 0.        ,
       0.55555556, 0.        , 0.66666667, 0.        , 0.77777778,
       0.        , 0.88888889, 0.11111111, 0.        , 0.11111111,
       0.33333333, 0.11111111, 0.77777778, 0.11111111, 0.88888889,
       0.22222222, 0.        , 0.22222222, 0.33333333, 0.22222222,
       0.88888889, 0.33333333, 0.        , 0.33333333, 0.33333333,
       0.33333333, 0.88888889, 0.44444444, 0.        , 0.44444444,
       0.33333333, 0.44444444, 0.44444444, 0.44444444, 0.77777778,
       0.44444444, 0.88888889, 0.55555556, 0.        , 0.5555

In [10]:
[task_text,reward_text]

['task:["(\'tomato\', \'steak\')", "(\'steak\',)"]', 'reward:[-0.1]']

In [6]:
(True or False) and True

True

In [1]:
import argparse
import os
import random
import time
from distutils.util import strtobool
import pygame
import gym
import numpy as np
from gym_macro_overcooked.macActEnvWrapper import MacEnvWrapper

rewardList = {
    "subtask finished": 10,
    "correct delivery": 200,
    "wrong delivery": -5,
    "step penalty": -0.1,
    "burned penalty": -2,
}

env = gym.make(
    "Dynamic-v3",
    n_task=2,
    rewardList=rewardList,
    n_agent=1,
    obs_radius=2,
    obs_mode="vector",
    debug=True,
)
env.reset()


# 将键盘WASD输入转换为动作的函数
def key2action(key):
    if key == "d":
        return 0
    elif key == "s":
        return 1
    elif key == "a":
        return 2
    elif key == "w":
        return 3
    else:
        return None




pygame 2.5.2 (SDL 2.28.3, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
env.tomato[0].chopped=True
env.plate[0].containing.append(env.tomato[0])
env.agent[0].holding=env.plate[0]

In [9]:
from ..items import Tomato, Lettuce, Plate, Knife, Delivery, Agent, Food


ImportError: attempted relative import with no known parent package

In [8]:
isinstance(env.agent[0].holding, Plate)

NameError: name 'Plate' is not defined

In [3]:
env.render()

array([[[114,  93,  51],
        [114,  93,  51],
        [114,  93,  51],
        ...,
        [114,  93,  51],
        [114,  93,  51],
        [114,  93,  51]],

       [[114,  93,  51],
        [220, 170, 110],
        [220, 170, 110],
        ...,
        [220, 170, 110],
        [220, 170, 110],
        [114,  93,  51]],

       [[114,  93,  51],
        [220, 170, 110],
        [220, 170, 110],
        ...,
        [220, 170, 110],
        [220, 170, 110],
        [114,  93,  51]],

       ...,

       [[114,  93,  51],
        [220, 170, 110],
        [220, 170, 110],
        ...,
        [220, 170, 110],
        [220, 170, 110],
        [114,  93,  51]],

       [[114,  93,  51],
        [220, 170, 110],
        [220, 170, 110],
        ...,
        [220, 170, 110],
        [220, 170, 110],
        [114,  93,  51]],

       [[114,  93,  51],
        [114,  93,  51],
        [114,  93,  51],
        ...,
        [114,  93,  51],
        [114,  93,  51],
        [114,  93,  51]]

: 

In [37]:
key=input("Please input a key: ")
action = key2action(key)
new_obs, reward, done, info = env.step([action])
print(f"Reward: {reward}, Done: {done}, Info: {info}")
env.render()

Reward: [-0.1], Done: False, Info: {'cur_mac': [3], 'mac_done': [True], 'collision': []}


array([[[114,  93,  51],
        [114,  93,  51],
        [114,  93,  51],
        ...,
        [114,  93,  51],
        [114,  93,  51],
        [114,  93,  51]],

       [[114,  93,  51],
        [220, 170, 110],
        [220, 170, 110],
        ...,
        [220, 170, 110],
        [220, 170, 110],
        [114,  93,  51]],

       [[114,  93,  51],
        [220, 170, 110],
        [220, 170, 110],
        ...,
        [220, 170, 110],
        [220, 170, 110],
        [114,  93,  51]],

       ...,

       [[114,  93,  51],
        [220, 170, 110],
        [220, 170, 110],
        ...,
        [220, 170, 110],
        [220, 170, 110],
        [114,  93,  51]],

       [[114,  93,  51],
        [220, 170, 110],
        [220, 170, 110],
        ...,
        [220, 170, 110],
        [220, 170, 110],
        [114,  93,  51]],

       [[114,  93,  51],
        [114,  93,  51],
        [114,  93,  51],
        ...,
        [114,  93,  51],
        [114,  93,  51],
        [114,  93,  51]]

In [31]:
if env.plate[0].containing:
    print('yes')

yes


In [36]:
if env.agent[0].holding.containing:
    print('yes')

yes
